# Author: Markus Laubenthal
This Notebook implements the STDenseNet with the attention layer.

In [ ]:
!wget "https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip" --no-verbose
!unzip spatio_merged_data_iss.zip
!rm spatio_merged_data_iss.zip

2021-01-29 21:00:50 URL:https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip [37282886/37282886] -> "spatio_merged_data_iss.zip" [1]
Archive:  spatio_merged_data_iss.zip
   creating: input/
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/grid_ML.geojson   
  inflating: __MACOSX/input/._grid_ML.geojson  
  inflating: input/internet_ML.csv   
  inflating: __MACOSX/input/._internet_ML.csv  
  inflating: input/satelite.png      
  inflating: __MACOSX/input/._satelite.png  
  inflating: input/weather.csv       
  inflating: __MACOSX/input/._weather.csv  
  inflating: input/social_pulse_ML.csv  
  inflating: __MACOSX/input/._social_pulse_ML.csv  


In [ ]:
!rm -rf functions
!git clone https://github.com/markuslaubenthal/lab_st.git functions
!cd functions && git checkout original_attention

Cloning into 'functions'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 543 (delta 164), reused 140 (delta 77), pack-reused 314
Receiving objects: 100% (543/543), 192.68 KiB | 19.27 MiB/s, done.
Resolving deltas: 100% (361/361), done.
Branch 'original_attention' set up to track remote branch 'original_attention' from 'origin'.
Switched to a new branch 'original_attention'


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, activations
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file
from keras.optimizers import SGD, Adam


In [ ]:
from functions.preprocessing.DataImport import load_and_scale_internet, load_and_scale_satelite, load_and_scale_social, load_and_scale_weather, create_space_invariant
from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file

In [ ]:
f = getFileHandler('training_data.h5')
internet, internet_origin, internet_min, internet_max = load_and_scale_internet('input/internet_ML.csv', f, log10=False, min_max_axis=1)


In [ ]:
x_closeness = generate_dataset(internet, [1,2,3], 168, f, "closeness")
x_period = generate_dataset(internet, [24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 167, 168], 168, f, "period")
y = generate_label(internet, 168, f)



In [ ]:
from functions.preprocessing.TestTrainSplit import seven_days_train_test_split

x_closeness_train, x_closeness_test = seven_days_train_test_split(x_closeness, 168)
x_period_train, x_period_test = seven_days_train_test_split(x_period, 168)
y_train, y_test = seven_days_train_test_split(y, 168)

In [ ]:
from functions.model.SplitDenseNetFactory import SplitDenseNetFactory
from keras.optimizers import SGD, Adam
dn = SplitDenseNetFactory()
model = dn.Model((x_closeness_train.shape[3], x_period_train.shape[3]), depth=8, growth_rate=4, initial_filters=8, time_embedding_method=None, t_minus_one=False)

KerasTensor(type_spec=TensorSpec(shape=(None, 10000, 17, 1), dtype=tf.float32, name=None), name='reshape_4/Reshape:0', description="created by layer 'reshape_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 10000, 17, 64), dtype=tf.float32, name=None), name='conv2d_5/BiasAdd:0', description="created by layer 'conv2d_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 10000, 1088), dtype=tf.float32, name=None), name='reshape_5/Reshape:0', description="created by layer 'reshape_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 10000, 1), dtype=tf.float32, name=None), name='multi_head_attention_1/attention_output/add:0', description="created by layer 'multi_head_attention_1'")


In [ ]:
def scheduler(epoch, lr):
  return lr / 10 ** (epoch // 10)


lr = 0.01
epochs = 16
model.compile(optimizer=Adam(lr=lr, decay= lr/float(epochs)),
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()],
              )


model.fit([x_closeness_train, x_period_train], y_train,
          validation_data=([x_closeness_test, x_period_test], y_test), epochs=epochs,
          batch_size=2
          )

Epoch 1/16
576/576 [==============================] - 161s 276ms/step - loss: 0.0034 - root_mean_squared_error: 0.0580 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0448
Epoch 2/16
576/576 [==============================] - 159s 275ms/step - loss: 0.0032 - root_mean_squared_error: 0.0568 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0448
Epoch 3/16
576/576 [==============================] - 158s 275ms/step - loss: 0.0033 - root_mean_squared_error: 0.0574 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0448
Epoch 4/16
576/576 [==============================] - 159s 275ms/step - loss: 0.0033 - root_mean_squared_error: 0.0578 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0448
Epoch 5/16
105/576 [====>.........................] - ETA: 2:04 - loss: 0.0033 - root_mean_squared_error: 0.0574

KeyboardInterrupt: ignored

In [ ]:
from functions.postprocessing.ErrorEvaluation import calculate_errors

In [ ]:
pred = model.predict([x_closeness[:], x_period[:]], batch_size=2)

pred, base = calculate_errors(pred, internet_origin, internet_min, internet_max, log10=False)

all:  88.94722284875151
test:  92.12589388420436
val:  62.96689531304112
